In [27]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from spacy.lang.ru.stop_words import STOP_WORDS

df_inter = pd.read_csv('./mod_data/interactions.csv')
df_items = pd.read_csv('./mod_data/items.csv')
df_users = pd.read_csv('./mod_data/users.csv')

## Items

In [28]:
df_items.sample(1)

,item_id,content_type,title,genres,age_rating,keywords
11514,15748,film,Как знать…,"драмы, мелодрамы, комедии",16,"любовный треугольник, бейсбол, 2010, соединенн..."


In [29]:
def tokenize(line: str):
    return [word for word in line.split(', ') 
            if not word.isnumeric() and word not in {'', 'nan'}]

keywords_vectorizer = TfidfVectorizer(
    tokenizer=tokenize, token_pattern=None,
    max_features=100, stop_words=list(STOP_WORDS))

X_keywords = keywords_vectorizer.fit_transform(
    df_items['keywords'].values.astype('U')).todense()

X_content_type = (df_items['content_type'] == 'film').\
    astype(int).values.reshape(-1, 1)

age_rating_encoder = OneHotEncoder(sparse_output=False)
X_age_rating = age_rating_encoder.fit_transform(
    df_items['age_rating'].values.astype('U').reshape(-1, 1))

genres_vectorizer = TfidfVectorizer(tokenizer=tokenize, token_pattern=None)
X_genres = genres_vectorizer.fit_transform(
    df_items['genres'].values.astype('U')).todense()

In [30]:
keywords_vectorizer.get_feature_names_out()

array(['автокатастрофа', 'бандиты', 'бар', 'беременность', 'близких',
       'блокбастер', 'боевые искусства', 'бой', 'больница', 'борьба',
       'брак', 'брызги крови', 'великобритания', 'вечеринка', 'взрыв',
       'война', 'врач', 'выживание', 'выстрел в голову',
       'выстрел в грудь', 'гангстеры', 'германия', 'героизм', 'героя',
       'деньги', 'держать на прицеле', 'детей', 'дети', 'дождь', 'дочери',
       'дружба', 'друзья', 'женская нагота', 'женщины', 'жены',
       'жестокость', 'жизненных', 'жизнь', 'загадки', 'задание',
       'замедленная сцена', 'застреленный', 'злом', 'италия', 'канада',
       'конфликт', 'конфликты', 'кровь', 'любовь', 'матери', 'месть',
       'миссия', 'мужа', 'мужчина с обнаженной грудью', 'мужчины',
       'название', 'обман', 'отношения', 'отношения мужа и жены',
       'отношениях', 'отца', 'отцы', 'паре', 'потеря', 'правопорядка',
       'преодоление', 'преследования', 'преступления', 'приоритетов',
       'проблемы', 'прошлое', 'путешестви

In [31]:
pd.Series(
    data=np.squeeze(np.asarray(X_keywords.sum(axis=0))),
    index=keywords_vectorizer.get_feature_names_out()
    ).sort_values(ascending=False)[:10]

россия                     1908.561897
соединенные штаты          1327.018417
франция                     934.118947
отношения                   878.104439
сша                         752.556432
дружба                      444.615631
соединенное королевство     437.047073
любовь                      433.247950
ссср                        347.287887
борьба                      338.713945
dtype: float64

In [32]:
X_items = np.hstack([X_content_type, X_genres, X_age_rating, X_keywords])
X_items.shape

(11909, 200)

## Users

In [33]:
df_users.sample(1)

,user_id,age,income,sex,kids_flg
446566,220600,age_18_24,income_40_60,Ж,1


In [38]:
age_encoder = OneHotEncoder(sparse_output=False)
X_age = age_encoder.fit_transform(
    df_users['age'].values.astype('U').reshape(-1, 1))

income_encoder = OneHotEncoder(sparse_output=False)
X_income = income_encoder.fit_transform(
    df_users['income'].values.astype('U').reshape(-1, 1))

X_sex = (df_users['sex'] == 'М').astype(int).values.reshape(-1, 1)

In [40]:
X_users = np.hstack([X_age, X_income, X_sex,
    df_users['kids_flg'].to_numpy().reshape(-1, 1)])
X_users.shape

(448798, 14)

In [41]:
np.savez_compressed('encoded_data', X_users=X_users, X_items=X_items)